In [10]:

import pandas as pd
import numpy as np
from skimage.measure import label, regionprops
import time

class Excel_Automation:

  def __init__ (self, path_file_excel, sheet_name = ""):
    self.path_file_excel = path_file_excel
    self.sheet_name = sheet_name
    if (sheet_name == ""):
      self.result = Excel_Automation.cleaning_table(self,self.path_file_excel)
    else:
      self.result = Excel_Automation.cleaning_table_by_sheetname(self,self.path_file_excel, self.sheet_name)

  #================ MAIN METHOD ================================================
  #=============================================================================

  #======= MAIN METHOD (cleaning excel include all sheet) =========
  def cleaning_table(self, data_path):

      xl = pd.ExcelFile(data_path)

      list_sheet = []

      start_time = time.time()
      # read excel file
      for i in xl.sheet_names:
            df = pd.read_excel(data_path, sheet_name=i, engine='openpyxl')
            size = df.shape
            if ((size[0] != 0) and (size[1] != 0)):
              list_sheet.append(df)

      print("---read_file %s seconds ---" % (time.time() - start_time))
      self.list_read = list_sheet
      # parsing table
      start_time = time.time()
      list_data = []
      for i in list_sheet:
        list_data.append(Excel_Automation.parse_tabel(i))

      self.result_parsing = list_data
      print("---parsing %s menit ---" % ((time.time() - start_time)/60))


      # remove None Row in table
      start_time = time.time()
      for i in list_data:  
        for index,y in enumerate(i):
          i[index] = Excel_Automation.remove_unused_row(y)  

      self.result_deleted_row = list_data
      print("---remove_unused_row %s menit ---" % ((time.time() - start_time)/60))

      # delete those that are not tables
      start_time1 = time.time()
      list_clean_parsing = []
      for i in list_data:
        temporary = []
        for y in i :
          check = Excel_Automation.is_it_table(y,1)
          if (check):
            temporary.append(y)
        list_clean_parsing.append(temporary)
      print("---checking table %s second ---" % ((time.time() - start_time)))

      #print(len(list_clean_parsing))
      # clean duplicate table
    
      start_time0 = time.time()
      list_data_nonDuplicate = []
      
      for i in list_clean_parsing:
        #print(len(i))
        temporary = []
        counter2 = 0
        list_duplicate = []
        for y,value in enumerate(i):
          #print(y)
          start_time1 = time.time()
          if (y not in list_duplicate):
            #print(1)
            for j in range(len(i)):
              if (j not in list_duplicate): 
                if (y != j and j > y):
                  #print(str(y)+", " +str(j))  
                  check_value = Excel_Automation.check_similarity_table_2(value, i[j])
                  if (check_value == 1):
                    #print(3)
                    #print(str(y)+", " +str(j))
                    list_duplicate.append(y)
                  elif (check_value == 2):
                    #print(4)
                    #print(str(y)+", " +str(j))
                    list_duplicate.append(j)
                  #print("===================")
                  #print("---loop %s menit ---" % ((time.time() - start_time1)/60))
          
          #print("")
          #print("=====================================================")
        #print(list_duplicate)
        for y in range(len(i)):
          if (y not in list_duplicate):
            temporary.append(i[y])          

        list_data_nonDuplicate.append(temporary)  

      self.data_non_duplicate = list_data_nonDuplicate
      print("---checking_duplicate_table %s menit ---" % ((time.time() - start_time0)/60))
      
      # remove title
      start_time = time.time()
      #for i in list_data_nonDuplicate:
      for i in list_data_nonDuplicate:
        for index,y in enumerate(i):
          i[index] = Excel_Automation.remove_title(y)
      
      self.removed_title = list_data_nonDuplicate
      print("---remove_title %s menit ---" % ((time.time() - start_time)/60))

      list_table = []
      for i in list_data_nonDuplicate:
        temporary = []
        for y in i:
          table = np.array(y.T.reset_index().values.T.tolist())
          if (Excel_Automation.is_it_table(table,1)):
            temporary.append(y)
        list_table.append(temporary)

      # check merged cell
      start_time = time.time()
      for i in list_table:
        for index,y in enumerate(i):
          i[index] = Excel_Automation.check_merged_cell(y)

      self.checked_merge_cell = list_table
      print("---check_merge %s menit ---" % ((time.time() - start_time)/60))

      list_table_result = []
      for i in list_table:
        temporary = []
        for y in i:
          table = np.array(y.T.reset_index().values.T.tolist())
          if (Excel_Automation.is_it_table(table,1)):
            temporary.append(y)
        list_table_result.append(temporary)

      return list_table_result

  #======= MAIN METHOD (cleaning excel by particular sheet)
  def cleaning_table_by_sheetname(self,data_path, sheet_Name):

      start_time = time.time()
      # read excel file
      df = pd.read_excel(data_path, sheet_name=sheet_Name, engine='openpyxl')
      print("---read_file %s seconds ---" % (time.time() - start_time))
      
      start_time = time.time()
      # parsing table
      list_data = Excel_Automation.parse_tabel(df)
      self.result_parsing = list_data
      print("---parsing %s menit ---" % ((time.time() - start_time)/60))

      start_time = time.time()
      # remove None Row in table
      for index,i in enumerate(list_data):
        list_data[index] = Excel_Automation.remove_unused_row(i)
      self.result_deleted_row = list_data
      print("---remove_unused_row %s menit ---" % ((time.time() - start_time)/60))

      start_time1 = time.time()
      # delete those that are not tables
      list_clean_parsing = []
      for i in list_data:
        check = Excel_Automation.is_it_table(i,1)
        if (check):
          list_clean_parsing.append(i)
      print("---checking table %s second ---" % ((time.time() - start_time)))

      # remove duplicate table
      list_data_nonDuplicate = [] 
      list_duplicate = []

      for y,value in enumerate(list_clean_parsing):
        if (y not in list_duplicate):
          for j in range(len(list_clean_parsing)):
            if (j not in list_duplicate):
              if (y != j and j > y):
                check_value = Excel_Automation.check_similarity_table_2(value, list_clean_parsing[j])
                if (check_value == 1):
                  list_duplicate.append(y)
                elif (check_value == 2):
                  list_duplicate.append(j)

      for i in range(len(list_clean_parsing)):
        if (i not in list_duplicate):
          list_data_nonDuplicate.append(list_clean_parsing[i])

      # remove title
      for index,y in enumerate(list_data_nonDuplicate):
        list_data_nonDuplicate[index] = Excel_Automation.remove_title(y)
      
      self.removed_title = list_data_nonDuplicate

      list_table = []
      for i in list_data_nonDuplicate:
        table = np.array(i.T.reset_index().values.T.tolist())
        if (Excel_Automation.is_it_table(table,1)):
          list_table.append(i)

      # cek merged cell
      for index,y in enumerate(list_table):
        list_table[index] = Excel_Automation.check_merged_cell(y)
      self.checked_merge_cell = list_table

      list_table_result = []
      for i in list_table:
        table = np.array(y.T.reset_index().values.T.tolist())
        if (Excel_Automation.is_it_table(table,1)):
          list_table_result.append(i)

      return list_table_result

  #====== METHOD FOR PARSING ===========
  def parse_tabel(data):
      data_array = np.array(data.T.reset_index().values.T.tolist())
      data_array_copy = data_array.copy()
      size = data_array.shape

      # convert table into 1 and 0
      for i in range(size[0]):
        for y in range(size[1]):
          if (str(data_array[i,y]) == 'nan' or 'Unnamed' in str(data_array[i,y])):
            data_array[i,y] = 0
          else:
            data_array[i,y] = 1


      list_of_dataframes = []
      l = label(data_array) # labeling tabel 

      region = regionprops(l) # get start and end of row and column

      #print(len(region))
      # parsing besed on region
      for i in range(len(region)):
        #print(i)
        firstRow = region[i].bbox[0]
        firstColumn = region[i].bbox[1]  

        if (i < (len(region) - 1)):
          secondRow = region[i+1].bbox[0]
          secondColumn = region[i+1].bbox[1]

          if (firstRow == secondRow):
            if (firstColumn == secondColumn):
              slice_data = data_array_copy[region[i].bbox[0]:region[i+1].bbox[2],region[i].bbox[1]:region[i+1].bbox[3]]
              list_of_dataframes.append(pd.DataFrame(slice_data[1:], columns = slice_data[0]))
            else:
              if (i < (len(region) - 2)):
                thirdRow = region[i+2].bbox[0]
                thirdColumn = region[i+2].bbox[1]
                if (thirdColumn == secondColumn and thirdRow == firstRow):
                  slice_data = data_array_copy[region[i].bbox[0]:region[i+2].bbox[2],region[i].bbox[1]:region[i+2].bbox[3]]
                  list_of_dataframes.append(pd.DataFrame(slice_data[1:], columns = slice_data[0]))
                else:
                  slice_data = data_array_copy[region[i].bbox[0]:region[i+1].bbox[2],region[i].bbox[1]:region[i+1].bbox[3]]
                  list_of_dataframes.append(pd.DataFrame(slice_data[1:], columns = slice_data[0]))
              else:
                slice_data = data_array_copy[region[i].bbox[0]:region[i+1].bbox[2],region[i].bbox[1]:region[i+1].bbox[3]]
                list_of_dataframes.append(pd.DataFrame(slice_data[1:], columns = slice_data[0]))
          else:
            slice_data = data_array_copy[region[i].bbox[0]:region[i].bbox[2],region[i].bbox[1]:region[i].bbox[3]]
            list_of_dataframes.append(pd.DataFrame(slice_data[1:], columns = slice_data[0]))
        else:
          slice_data = data_array_copy[region[i].bbox[0]:region[i].bbox[2],region[i].bbox[1]:region[i].bbox[3]]
          list_of_dataframes.append(pd.DataFrame(slice_data[1:], columns = slice_data[0]))

      list_parseTable = []

      # checking is it table or not
      for i in list_of_dataframes:
        print
        if (Excel_Automation.is_it_table(i,1)):
            list_parseTable.append(i)

      # remove table that contains none value in all cell (error in parsing)
      list_cleanParsing = []
      for i in list_parseTable:
        if (Excel_Automation.remove_none_table(i) == False):
          list_cleanParsing.append(i)
      return list_cleanParsing
  #=============================================================================
  #=============================================================================
  '''
  def delete_similarity_table(list_data, new_data):
    start_time = time.time()
    list_data_result = list_data.copy()

    if (len(list_data) == 0):
      list_data_result.append(new_data)
    else:
      for i,value in enumerate(list_data):
        check_value = Excel_Automation.check_similarity_table_2(new_data, value)
        if (check_value == 1):
          break           
        elif (check_value == 2):
          list_data_result.pop(i)
          if (i ==  len(list_data)-1):
            list_data_result.append(new_data)
        else:
          if (i ==  len(list_data)-1):
            list_data_result.append(new_data)
    print("---ceking duplicate %s menit ---" % ((time.time() - start_time)/60))
    print("=================================================================")
    print("")
    return list_data_result
    '''
  #================== REMOVE UNUSED ROW ========================================
  #=============================================================================

  #======== METHOD FOR CHECKING COLUMN INDEX ==========
  def get_index_column(data):
    list_index_column = []

    # convert value in list data (variable data) to integer
    for i in data:
      try:
        a = int(i)
        list_index_column.append(a)
      except:
        continue
    result = 0
    
    dif_value = []
    for i in range(len(list_index_column)-1):
      dif_value.append(list_index_column[i]-list_index_column[i+1])

    if (len(list_index_column) <= 2): 
      result = False
    elif (Excel_Automation.check_zero_dif(dif_value)):
      result = True
    else:
      different_value = list_index_column[1] - list_index_column[0]
      
      counter = 0
      for i in range(len(list_index_column)-1):
        check = list_index_column[i+1] - list_index_column[i]
        if (check != different_value):
          result = False
          break
        else:
          if ((i+1) == len(list_index_column)-1):
            if (list_index_column[0] >= 1500):
              result = False
            else:
              result = True

    return result

  #============== METHOD FOR CHECKING INDEX COLUMN THAT HAVE DIFFERENT VALUES 0 ===================
  def check_zero_dif(data):
    list_boolean = []
    for i in data:
      if (i == 0):
        list_boolean.append(True)
      else:
        list_boolean.append(False)

    if (False in list_boolean):
      return False
    else:
      return True

  #======== METHOD FOR CHECKING NONE ROW ==========
  def get_none_row(data):
    list_noneData = []
    counter = 0
    
    for i in data:
      if (str(i) == 'nan' or 'Unnamed' in str(i)):
        list_noneData.append(counter)
      counter += 1
    
    if (len(list_noneData) ==  len(data)):
      return True
    else:
      return False

  #========== METHOD FOR DELETE NONE ROW ===========
  def remove_unused_row(data):
    print(data)
    data_array = np.array(data.T.reset_index().values.T.tolist())
    data_result = []

    for i in range(len(data_array)):
      if (Excel_Automation.get_none_row(data_array[i,:]) or Excel_Automation.get_index_column(data_array[i,:])):
        continue
      else:
        data_result.append(data_array[i,:])

    result = np.array(data_result)

    return pd.DataFrame(result[1:], columns = result[0])

  #=========== METHOD FOR CHECKING NONE TABLE AND DELETE IT
  def remove_none_table(table):
    data_array = np.array(table.T.reset_index().values.T.tolist())

    row = data_array.shape[0]
    column = data_array.shape[1]

    counter = 0
    for i in range(row):
      for y in range(column):
        if (data_array[i,y] == None or str(data_array[i,y]) == 'nan' or str(data_array[i,y]) == 'None' or 'Unnamed' in str(data_array[i,y])):
          counter += 1
    
    if (counter == (row*column)):
      return True
    else:
      return False

  #=============================================================================
  #=============================================================================

  #==================== CHECK DUPLICATE TABLE ==================================
  #=============================================================================

  #========== METHOD FOR CHECKING TABLE SIMILARITY BASED ON DATA AT THE TABLE ===============
  def check_similarity_table_2(table1, table2):
    #start = time.time()
    np_array1 = np.array(table1.T.reset_index().values.T.tolist())
    np_array2 = np.array(table2.T.reset_index().values.T.tolist())

    size_1 = np_array1.shape[0]*np_array1.shape[1]
    size_2 = np_array2.shape[0]*np_array2.shape[1]

    #0 brarti tidak ada yang sama
    #1 brarti table_2 in table_1
    #2 brarti table_1 in table_2

    result = 0
    
    if (size_1 > size_2):
      #check = np.isin(np_array1, np_array2).sum()
      check = Excel_Automation.check_detail_similarity(np_array2,np_array1)
      #if (check == size_2 or check > size_2):
      if (check == 1):
        result = 2
      else:
        result = 0
    elif (size_2 > size_1):
      #check = np.isin(np_array2, np_array1).sum()
      check = Excel_Automation.check_detail_similarity(np_array1,np_array2)
      #if (check == size_1 or check > size_1):
      if (check == 1):
        result = 1
      else:
        result = 0
    else:
      check = Excel_Automation.check_detail_similarity(np_array1,np_array2)
      #check = np.isin(np_array2, np_array1).sum()
      #if (check == size_1):
      if (check == 1):
        result = 1
      else:
        result = 0

    return result

  def check_detail_similarity(table_1, table_2):
    start_value = table_1[0,0]
    size = table_1.shape[1]*table_1.shape[0]
    loc = np.where(table_2 == start_value)
    if (len(loc[0]) == 0 ):
      return 0
    else:
      list_result = []
      for i in range(len(loc[0])):
        selec_data = table_2[loc[0][i]:(loc[0][i]+table_1.shape[0]),loc[1][i]:(loc[1][i]+table_1.shape[1])]
        check_size = selec_data.shape[1]*selec_data.shape[0]
        if (check_size != size):
          list_result.append(False)
        else:
          check_data = np.isin(table_1, selec_data).sum()
          if (check_data == size):
            list_result.append(True)
          else:
            list_result.append(False)
      if (True in list_result):
        return 1
      else:
        return 0
    '''
    if (size_1 > size_2):
      result = False
    else:
      check = np.isin(np_array2, np_array1).sum()
      if (check == size_1 or check > size_1):
        result = True
      else:
        result = False
    #print("---cek duplicate %s menit ---" % ((time.time() - start)/60))

    return result
    '''
  '''
  def check_similarity_data_table(table1, table2):
    np_array1 = np.array(table1.T.reset_index().values.T.tolist())
    np_array2 = np.array(table2.T.reset_index().values.T.tolist())

    rowSize1 = np_array1.shape[0]
    columnSize1 = np_array1.shape[1]
    rowSize2 = np_array2.shape[0]
    columnSize2 = np_array2.shape[1]
    
    number_true_value = 0
    counter = 0

    # compare data table in table1 to table2 one by one
    for i in range(rowSize1):
      for j in range(columnSize1):
        loc = np.where(np_array2 == np_array1[i,j])
        if (len(loc[0]) != 0):
          counter += 1

    result = counter / (rowSize1*columnSize1)
    if (result == 1):
      return True
    else: 
      return False

  #=========== METHOD FOR CHECKING TABLE SIMILARITY BASED ON SHAPE AND START VALUE ============
  def check_similarity_table(table1, table2):
    np_array1 = np.array(table1.T.reset_index().values.T.tolist())
    np_array2 = np.array(table2.T.reset_index().values.T.tolist())

    check_1 = 0

    row_size_1 = np_array1.shape[0]
    column_size_1 = np_array1.shape[1]
    row_size_2 = np_array2.shape[0]
    column_size_2 = np_array2.shape[1]

    #===== check table1 in table2
    if (np_array2.shape[0] >= np_array1.shape[0]):
      if (np_array1.shape[1] > np_array2.shape[1]):
        check_1 = 0
      elif ((np_array1.shape[1] == np_array2.shape[1]) and (np_array1.shape[0] == np_array2.shape[0])):
        if  (Excel_Automation.check_similarity_data_table(table1, table2)):
          check_1 = 2
        else:
          check_1 = 0

      else:
        start_value = np_array1[0,0]
        loc = np.where(np_array2 == start_value)
        for i in range(len(loc[0])):
          start_row = loc[0][i]
          start_column = loc[1][i]
          selected_array = np_array2[start_row:(start_row + row_size_1),start_column:(start_column + column_size_1)]
          row_selected_array = selected_array.shape[0]
          column_selected_array = selected_array.shape[1]        
          if ((row_size_1 == row_selected_array) and (column_size_1 == column_selected_array)):
            check_1 = 1
          else:
            check_1 = 0

    return check_1
  #=============================================================================
  #=============================================================================
  '''
  #========== METHOD FOR REMOVING TITLE ========================================
  #=============================================================================
  def remove_title(data):
    result = []
    data_array = np.array(data.T.reset_index().values.T.tolist())
    check_title, list_index = 0, 0

    counter = 0
    for i in range(len(data_array)):
      check_title,lis_index = Excel_Automation.check_none_data(data_array[i,:], (len(data_array[i,:])))

      if (check_title):
        if (i < (len(data_array)-1)):
          if (Excel_Automation.matching_row(data_array[i,:], data_array[i+1,:], len(data_array[i,:])-1)):
            if (check_title):
              continue
            else:
              result.append(data_array[i,:])
          else:
            if (i < (len(data_array)-2)):
              if (Excel_Automation.matching_row_3Layer(data_array[i,:], data_array[i+1,:], data_array[i+2,:], len(data_array[i,:])-1)):
                if (check_title):
                  continue
                else:
                  result.append(data_array[i,:])
              else:
                if (i < (len(data_array)-3)):
                  if (Excel_Automation.matching_row_4Layer(data_array[i,:], data_array[i+1,:], data_array[i+2,:], data_array[i+3,:], len(data_array[i,:])-1)):
                    if (check_title):
                      continue
                    else:
                      result.append(data_array[i,:])
                else:
                  continue
            else:
              continue
        else:
          continue
      else:
        result.append(data_array[i,:])
      counter += 1

    return pd.DataFrame(result[1:], columns = result[0])

  #=========== METHOD FOR CHECK TITLE BASED ON NONE VALUE ============
  def check_none_data(list_data, length):
    counter = 0
    list_index = []
    data = 0

    list_index_nonNull = []
    for i in list_data:
      if (str(i) == 'nan' or 'Unnamed' in str(i)):
        list_index.append(counter)
      else:
        list_index_nonNull.append(counter)
      counter += 1

    boolean_result = 0
    
    if ((length-1) in list_index and len(list_data) > 3):
      if ((list_index[-1]-list_index[0]+1) == len(list_index) and (len(list_index_nonNull) > 1)):
        boolean_result = False
      elif ((list_index[-1]-list_index[0]+1) == len(list_index)):
        boolean_result = True
      else:
        boolean_result = False
    elif ((0 in list_index) and len(list_data) > 3):
      if ((list_index[-1]-list_index[0]+1) == len(list_index) and (len(list_index_nonNull) == 1)):
        boolean_result = True
      else:
        boolean_result = False
    else:
      boolean_result = False
    
    return boolean_result, list_index
  #=============================================================================
  #=============================================================================

  #============ METHOD FOR CHECKING MERGE CELL =================================
  #=============================================================================
  def check_merged_cell(data):
    data_array = np.array(data.T.reset_index().values.T.tolist())

    if (len(data_array) > 2):
      if (Excel_Automation.matching_row(data_array[0,:],data_array[1,:], len(data_array[0,:])-1)):

        data_array[0,:] = Excel_Automation.rename_header(data_array[0,:], data_array[1,:])
      
        data_result = np.delete(data_array,1,0)
        return pd.DataFrame(data_result[1:], columns = data_result[0])

      else:
        if (len(data_array) > 3):
          if (Excel_Automation.matching_row_3Layer(data_array[0,:],data_array[1,:], data_array[2,:], len(data_array[0,:])-1)):

            data_array[1,:] = Excel_Automation.rename_header(data_array[1,:], data_array[2,:])
            data_array[0,:] = Excel_Automation.rename_header(data_array[0,:], data_array[1,:])
      
            data_result = np.delete(data_array,[1,2],0)
            return pd.DataFrame(data_result[1:], columns = data_result[0])

          else:
            if (len(data_array) > 4):
              if (Excel_Automation.matching_row_4Layer(data_array[0,:],data_array[1,:], data_array[2,:], data_array[3,:], len(data_array[0,:])-1)):

                data_array[2,:] = Excel_Automation.rename_header(data_array[2,:], data_array[3,:])
                data_array[1,:] = Excel_Automation.rename_header(data_array[1,:], data_array[2,:])
                data_array[0,:] = Excel_Automation.rename_header(data_array[0,:], data_array[1,:])
      
                data_result = np.delete(data_array,[1,2,3],0)
                return pd.DataFrame(data_result[1:], columns = data_result[0])

              else:
                if (Excel_Automation.check_column_merge(data_array[0,:],data_array[1,:])[0]):
                  index = Excel_Automation.check_column_merge(data_array[0,:],data_array[1,:])[1]
                  counter = 1
                  for i in index:
                    data_array[0,i] = str(data_array[0,i-1]) + "_" + str(counter)
                    counter += 1
      
                  data_array[0,:] = Excel_Automation.rename_header(data_array[0,:], data_array[1,:])
      
                  data_result = np.delete(data_array,1,0)
                  return pd.DataFrame(data_result[1:], columns = data_result[0])

                else: 
                  return pd.DataFrame(data_array[1:], columns = data_array[0])
            else:
              return pd.DataFrame(data_array[1:], columns = data_array[0])
        else:
          return pd.DataFrame(data_array[1:], columns = data_array[0])
    else:
      return pd.DataFrame(data_array[1:], columns = data_array[0])

  #========== METHOD FOR CHECKING COLUMN MERGE IN HEADER ==============
  def check_column_merge(data1, data2):
    None1, nonNull1 = Excel_Automation.get_none_index(data1)
    None2, nonNull2 = Excel_Automation.get_none_index(data2)

    list_index_nonCover = []
    for i in None1:
      if (i in None2):
        list_index_nonCover.append(i)
    
    index_merge = []
    for i in None1:
      if (i in nonNull2):
        continue
      else:
        index_merge.append(i)

    list_column_merge = []
    for i in index_merge:
      if (i in list_index_nonCover):
        list_column_merge.append(i)

    if (len(list_column_merge) == 0):
      return False,0
    else:
      return True,list_column_merge

  #=========== METHOD FOR CHECKING TWO ROW ARE MERGE IN HEADER ========
  def matching_row(data1, data2, last_index):
    None1, nonNull1 = Excel_Automation.get_none_index(data1)
    None2, nonNull2 = Excel_Automation.get_none_index(data2)
    list_boolean = []

    list_duplicate_index = []
    list_index_nonCover = []

    for i in nonNull1:
      if (i in nonNull2):
        list_duplicate_index.append(i)
    
    if (0 in list_duplicate_index or last_index in list_duplicate_index):
      list_boolean.append(False)
    else:
      for i in list_duplicate_index:
        if (i == 0):
          if ((i+ 1) in list_duplicate_index):
            list_boolean.append(False)
          else:
            list_boolean.append(True)
        elif (i == last_index):
          if ((i-1) in list_duplicate_index):
            list_boolean.append(False)
          else:
            list_boolean.append(True)
        else:
          if ((i-1) in list_duplicate_index or (i+1) in list_duplicate_index):
            list_boolean.append(False)
          else:
            list_boolean.append(True)
    
    cek1 = 0
    for i in None1:
      if (i in nonNull2):
        cek1 = True
      else:
        cek1 = False
    
    if (cek1):
      list_boolean.append(True)
    else:
      list_boolean.append(False)
    
    for i in None2:
      if (i in nonNull1):
        cek1 = True
      else:
        cek1 = False
    
    if (cek1):
      list_boolean.append(True)
    else:
      list_boolean.append(False)

    if (False in list_boolean):
      return False
    else:
      return True  

  #=========== METHOD FOR CHECKING THREE ROW ARE MERGE IN HEADER ========
  def matching_row_3Layer(data1, data2, data3, last_index):
    None1, nonNull1 = Excel_Automation.get_none_index(data1)
    None2, nonNull2 = Excel_Automation.get_none_index(data2)
    None3, nonNull3 = Excel_Automation.get_none_index(data3)  

    list_index_duplicate = []
    list_index_nonCover = []
    list_boolean = []

    for i in None2:
      if (i in None3):
        list_index_nonCover.append(i)

    for i in nonNull2:
      if (i in nonNull3):
        list_index_duplicate.append(i)

    list_index_inAllThree = []
    for i in list_index_duplicate:
      if (i in nonNull1):
        list_index_inAllThree.append(i)

  # mathcing row 2 layer
    if (0 in list_index_duplicate or last_index in list_index_duplicate):
      list_boolean.append(False)
    else:
      for i in list_index_duplicate:
        if (i == 0):
          if ((i+ 1) in list_index_duplicate):
            list_boolean.append(False)
          else:
            list_boolean.append(True)
        elif (i == last_index):
          if ((i-1) in list_index_duplicate):
            list_boolean.append(False)
          else:
            list_boolean.append(True)
        else:
          if ((i-1) in list_index_duplicate or (i+1) in list_index_duplicate):
            list_boolean.append(False)
          else:
            list_boolean.append(True)

  # mathcing row 3 layer
    if (0 in list_index_inAllThree or last_index in list_index_inAllThree):
      list_boolean.append(False)
    else:  
      for i in list_index_inAllThree:
        if (i == 0):
          if (i+1 in list_index_inAllThree):
            list_boolean.append(False)
          else:
            list_boolean.append(True)
        elif (i == last_index):
          if (i-1 in list_index_inAllThree):
            list_boolean.append(False)
          else:
            list_boolean.append(True)
        else:
          if (i-1 in list_index_inAllThree  or i+1 in list_index_inAllThree):
            list_boolean.append(False)
          else:
            list_boolean.append(True)
    
    for i in list_index_nonCover:
      if (i in None1):
        list_boolean.append(False)
      else:
        list_boolean.append(True)
    
    if (False in list_boolean):
      return False
    else:
      return True

  #=========== METHOD FOR CHECKING FOUR ROW ARE MERGE IN HEADER ========
  def matching_row_4Layer (data1, data2, data3, data4, last_index):
    None1, nonNull1 = Excel_Automation.get_none_index(data1)
    None2, nonNull2 = Excel_Automation.get_none_index(data2)
    None3, nonNull3 = Excel_Automation.get_none_index(data3)
    None4, nonNull4 = Excel_Automation.get_none_index(data4)

    list_index_duplicate = []
    list_index_inAllThree = []
    list_index_inAllFour = []
    list_index_nonCover1 = []
    list_index_nonCover2 = []
    list_boolean = []
    
    for i in None3:
      if (i in None4):
        list_index_nonCover1.append(i)    

    for i in list_index_nonCover1:
      if (i in None2):
        list_index_nonCover2.append(i)
    
    for i in nonNull3:
      if (i in nonNull4):
        list_index_duplicate.append(i)

    for i in list_index_duplicate:
      if (i in nonNull2):
        list_index_inAllThree.append(i)

    for i in list_index_inAllThree:
      if (i in nonNull1):
        list_index_inAllFour.append(i) 

  # mathcing row 2 layer
    if (0 in list_index_duplicate or last_index in list_index_duplicate):
      list_boolean.append(False)
    else:
      for i in list_index_duplicate:
        if (i == 0):
          if ((i+ 1) in list_index_duplicate):
            list_boolean.append(False)
          else:
            list_boolean.append(True)
        elif (i == last_index):
          if ((i-1) in list_index_duplicate):
            list_boolean.append(False)
          else:
            list_boolean.append(True)
        else:
          if ((i-1) in list_index_duplicate or (i+1) in list_index_duplicate):
            list_boolean.append(False)
          else:
            list_boolean.append(True)

  # mathcing row 3 layer
    if (0 in list_index_inAllThree or last_index in list_index_inAllThree):
      list_boolean.append(False)
    else:  
      for i in list_index_inAllThree:
        if (i == 0):
          if (i+1 in list_index_inAllThree):
            list_boolean.append(False)
          else:
            list_boolean.append(True)
        elif (i == last_index):
          if (i-1 in list_index_inAllThree):
            list_boolean.append(False)
          else:
            list_boolean.append(True)
        else:
          if (i-1 in list_index_inAllThree  or i+1 in list_index_inAllThree):
            list_boolean.append(False)
          else:
            list_boolean.append(True)

  # mathcing row 4 layer
    if (0 in list_index_inAllFour or last_index in list_index_inAllFour):
      list_boolean.append(False)
    else:  
      for i in list_index_inAllFour:
        if (i == 0):
          if (i+1 in list_index_inAllFour):
            list_boolean.append(False)
          else:
            list_boolean.append(True)
        elif (i == last_index):
          if (i-1 in list_index_inAllFour):
            list_boolean.append(False)
          else:
            list_boolean.append(True)
        else:
          if (i-1 in list_index_inAllFour or i+1 in list_index_inAllFour):
            list_boolean.append(False)
          else:
            list_boolean.append(True)

    for i in list_index_nonCover2:
      if (i in None1):
        list_boolean.append(False)
      else:
        list_boolean.append(True)

    if (False in list_boolean):
      return False
    else:
      return True  

  #========= METHOD FOR GET INDEX FOR NONE VALUE ==========
  def get_none_index(list_data):
    counter = 0 
    list_index_none = []
    list_index_nonNull = []
    for i in list_data:
      if (str(i) == 'nan' or 'Unnamed' in str(i)):
        list_index_none.append(counter)
      else:
        list_index_nonNull.append(counter)
      counter += 1
    return list_index_none, list_index_nonNull

  #=========== METHOD FOR RENAME TABLE HEADER AFTER CHECKING MERGE CELL ==============
  def rename_header(data1, data2):
    index_null1, index_nonNull1 = Excel_Automation.get_none_index(data1)
    index_null2, index_nonNull2 = Excel_Automation.get_none_index(data2)

    duplicate_index = []
    data = data1.copy()

    for i in index_nonNull1:
      if (i in index_nonNull2):
        duplicate_index.append(i)

    for i in range(len(data1)):
      if (i in duplicate_index):
        data[i] = str(data2[i]) + "_" + str(data1[i]) 
      elif ((i in index_null1) and (i in index_null2)):
        continue
      elif (i in index_null1):
        index = Excel_Automation.find_index_merged_cell(duplicate_index, i)
        data[i] = str(data2[i]) + "_" + str(data1[index])
      else:
        continue
    
    return data

  #========== METHOD FOR HELPING IN RENAME TABLE HEADER =========
  def find_index_merged_cell(duplicate_index, index):
    for i in range(len(duplicate_index)):
      if (index == duplicate_index[i]):
        return index
        break
      else:
        if (i != (len(duplicate_index)-1)):
          if (index > duplicate_index[i] and index < duplicate_index[i+1]):
            return duplicate_index[i]
          else:
            continue
        else:
          if (index > duplicate_index[i]):
            return duplicate_index[i]

  #=============================================================================
  #=============================================================================

  #====== METHOD FOR CHECK IT TABLE OR NOT =====================================
  #=============================================================================
  def is_it_table(data, max_row):
      row,column = data.shape

      if (row < max_row):
        return False
      elif (row <= 2 and column <= 2):
        return False
      else:
        return True

In [38]:
!pip install ipython-autotime
%load_ext autotime

time: 227 µs (started: 2021-03-10 07:15:03 +00:00)


In [ ]:
excel = Excel_Automation('/content/pasien.xlsx', 'Sheet1')

---read_file 15.955317974090576 seconds ---
---parsing 0.21293371121088664 menit ---
---remove_unused_row 0.12672767639160157 menit ---
---checking table 7.604484796524048 second ---
time: 3min 39s (started: 2021-03-09 08:02:41 +00:00)


In [ ]:
print(len(excel.result))

1
time: 1.46 ms (started: 2021-03-09 08:31:59 +00:00)


In [ ]:

xl = pd.ExcelFile('/content/SPI Desember 2020.xlsx')

list_sheet = []
for i in xl.sheet_names:
  df = pd.read_excel(data_path, sheet_name=i, engine='openpyxl')
  size = df.shape
  if ((size[0] != 0) and (size[1] != 0)):
    list_sheet.append(df)


BadZipFile: ignored

time: 24 ms (started: 2021-03-09 07:40:25 +00:00)


In [ ]:
print(len(excel.result[0]))
excel.result[0][0]

1


,No,Rekam Medis,Nama,Jenis Kelamin,Golongan Darah,Tanggal Lahir,Alamat Jalan,No Telepon,Status Pernikahan,Pendidikan Terakhir,Pekerjaan,Profesi,Agama,keterangan,duplicate
0,nan,024158,TUTIK SETYONINGSIH,P,nan,1964-12-31,PURWOSUMAN RT 13 PURWOSUMAN SIDOHARJO SRAGEN,nan,nan,nan,nan,nan,nan,nan,False
1,nan,024162,TN.KARIYONO,L,nan,1930-12-30,DUKUH RT 27 PATIHAN,08132800982727,nan,nan,nan,nan,Islam,nan,False
2,nan,029174,NY.SUMARTINI,2,nan,0000-00-00,KEDUNG WADUK RT 2 GEBANG,nan,nan,nan,nan,nan,Islam,invalid data type on column Jenis Kelamin. Sho...,False
3,nan,021686,SANTI SETYOWATI,P,nan,1988-12-31,KLECO WETAN RT 008 SIDOHARJO SIDOHARJO SRAGEN,nan,nan,nan,nan,nan,nan,nan,False
4,nan,024145,SARWINI,P,nan,1980-12-31,KEYONGAN RT 016 NGROMBO PLUPUH SRAGEN,nan,nan,nan,nan,nan,nan,nan,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50727,nan,74903,NY.MIJEM,P,nan,1935-11-21,SEPAT 35/08 SEPAT,02716882250,nan,nan,nan,nan,Islam,nan,False
50728,nan,75823,AN. ABIEL REYLEA HANZEL,L,nan,2018-07-17,PENGKOK RT 18,nan,nan,nan,nan,nan,Islam,nan,False
50729,nan,76460,suyanto,L,nan,1971-10-08,seketeng rt 03/15 kaliwuluh,nan,nan,nan,nan,nan,Islam,nan,False
50730,nan,78625,JOKO PRAYITNO,L,nan,1986-11-01,NGIJO RT 17,nan,nan,nan,nan,nan,Islam,nan,False


In [ ]:

excel.result_deleted_row[0][2]

,NO,NAMA LENGKAP (tanpa gelar),"TEMPAT, TANGGAL LAHIR",JENIS KELAMIN (Pria / Wanita),NIK,PENDIDIKAN TERAKHIR,NAMA LPQ,"JENIS LPQ (PAUDQu, TPQ, TQA, RTQ, PPTQ)",MASA KERJA,ALAMAT SESUAI KTP,nan,nan,nan
0,nan,nan,nan,nan,nan,nan,nan,nan,nan,DESA/KEL.,KECAMATAN,KABUPATEN / KOTA,PROVINSI


time: 33.5 ms (started: 2021-03-08 08:46:43 +00:00)


In [50]:
import pandas as pd
import numpy as np
from skimage.measure import label, regionprops
import time
import re

class Excel_Automation:

  def __init__ (self, path_file_excel, sheet_name = "", list_function = ""):
    self.path_file_excel = path_file_excel
    self.sheet_name = sheet_name
    if (sheet_name == ""):
      if (list_function == ""):
        self.result = Excel_Automation.cleaning_table(self,self.path_file_excel)
      else:
        self.result = Excel_Automation.cleaning_table(self,self.path_file_excel, list_function)
    else:
      if (list_function == ""):
        self.result = Excel_Automation.cleaning_table_by_sheetname(self,self.path_file_excel, self.sheet_name)
      else:
        self.result = Excel_Automation.cleaning_table_by_sheetname(self,self.path_file_excel, self.sheet_name, list_function)

  #================ MAIN METHOD ================================================
  #=============================================================================

  #======= MAIN METHOD (cleaning excel include all sheet) =========
  def cleaning_table(self, data_path, list_function=""):

      xl = pd.ExcelFile(data_path)

      list_sheet = []
      function = []

      if (list_function != ""):
        function = re.split('[\,\[\]]',list_function)
        function = [i for i in function if i != '']

      # read excel file
      for i in xl.sheet_names:
            df = pd.read_excel(data_path, sheet_name=i, engine='openpyxl')
            size = df.shape
            if ((size[0] != 0) and (size[1] != 0)):
              list_sheet.append(df)

      self.list_read = list_sheet

      # parsing table
      list_data = []

      for i in list_sheet:
        list_data.append(Excel_Automation.parse_tabel(i))

      self.result_parsing = list_data

      # remove None Row in table
      if (len(function) != 0 and 'remove_unused_row' in function):
        for i in list_data:  
          for index,y in enumerate(i):
            i[index] = Excel_Automation.remove_unused_row(y,status = True)
      else:
        for i in list_data:  
          for index,y in enumerate(i):
            i[index] = Excel_Automation.remove_unused_row(y,status = False)  

      self.result_deleted_row = list_data

      # delete those that are not tables
      list_clean_parsing = []
      for i in list_data:
        temporary = []
        for y in i :
          check = Excel_Automation.is_it_table(y,1)
          if (check):
            temporary.append(y)
        list_clean_parsing.append(temporary)

      # clean duplicate table
      list_data_nonDuplicate = []
      if (len(function) != 0 and 'check_duplicate_table' in function):
        for i in list_clean_parsing:
          temporary = []
          counter2 = 0
          list_duplicate = []
          for y,value in enumerate(i):
            if (y not in list_duplicate):
              for j in range(len(i)):
                if (j not in list_duplicate): 
                  if (y != j and j > y):
                    check_value = Excel_Automation.check_similarity_table_2(value, i[j])
                    if (check_value == 1):
                      list_duplicate.append(y)
                    elif (check_value == 2):
                      list_duplicate.append(j)

          for y in range(len(i)):
            if (y not in list_duplicate):
              temporary.append(i[y])          

          list_data_nonDuplicate.append(temporary)  
      else:
        list_data_nonDuplicate = list_clean_parsing

      self.data_non_duplicate = list_data_nonDuplicate
      
      # remove title
      if (len(function) != 0 and 'remove_title' in function):
        for i in list_data_nonDuplicate:
          for index,y in enumerate(i):
            i[index] = Excel_Automation.remove_title(y, status = True)
      else:
        for i in list_data_nonDuplicate:
          for index,y in enumerate(i):
            i[index] = Excel_Automation.remove_title(y, status = False)
      
      self.removed_title = list_data_nonDuplicate

      list_table = []
      for i in list_data_nonDuplicate:
        temporary = []
        for y in i:
          table = np.array(y.T.reset_index().values.T.tolist())
          if (Excel_Automation.is_it_table(table,1)):
            temporary.append(y)
        list_table.append(temporary)

      # check merged cell
      for i in list_table:
        for index,y in enumerate(i):
          i[index] = Excel_Automation.check_merged_cell(y)

      self.checked_merge_cell = list_table

      list_table_result = []
      for i in list_table:
        temporary = []
        for y in i:
          table = np.array(y.T.reset_index().values.T.tolist())
          if (Excel_Automation.is_it_table(table,1)):
            temporary.append(y)
        list_table_result.append(temporary)

      return list_table_result

  #======= MAIN METHOD (cleaning excel by particular sheet)
  def cleaning_table_by_sheetname(self,data_path, sheet_Name, list_function = ""):

      function = []

      if (list_function != ""):
        function = re.split('[\,\[\]]',list_function)
        function = [i for i in function if i != '']

      # read excel file
      df = pd.read_excel(data_path, sheet_name=sheet_Name, engine='openpyxl')
      
      # parsing table
      if (len(function) != 0 and 'parse_table' in function):
        list_data = Excel_Automation.parse_tabel(df)
      else:
        list_data = df

      self.result_parsing = list_data

      # remove None Row in table
      if (len(function) != 0 and 'remove_unused_row' in function):
        for index,i in enumerate(list_data):
          list_data[index] = Excel_Automation.remove_unused_row(i, status = True)
      else:
        for index,i in enumerate(list_data):
          list_data[index] = Excel_Automation.remove_unused_row(i, status = False)

      self.result_deleted_row = list_data

      # delete those that are not tables
      list_clean_parsing = []
      for i in list_data:
        check = Excel_Automation.is_it_table(i,1)
        if (check):
          list_clean_parsing.append(i)

      # remove duplicate table
      list_data_nonDuplicate = [] 
      if (len(function) != 0 and 'check_duplicate_table' in function):
        list_duplicate = []

        for y,value in enumerate(list_clean_parsing):
          if (y not in list_duplicate):
            for j in range(len(list_clean_parsing)):
              if (j not in list_duplicate):
                if (y != j and j > y):
                  check_value = Excel_Automation.check_similarity_table_2(value, list_clean_parsing[j])
                  if (check_value == 1):
                    list_duplicate.append(y)
                  elif (check_value == 2):
                    list_duplicate.append(j)

        for i in range(len(list_clean_parsing)):
          if (i not in list_duplicate):
            list_data_nonDuplicate.append(list_clean_parsing[i])
      else:
        list_data_nonDuplicate = list_clean_parsing

      self.data_non_duplicate = list_data_nonDuplicate

      # remove title
      if (len(function) != 0 and 'remove_title' in function):
        for index,y in enumerate(list_data_nonDuplicate):
          list_data_nonDuplicate[index] = Excel_Automation.remove_title(y, status = True)
      else:
        for index,y in enumerate(list_data_nonDuplicate):
          list_data_nonDuplicate[index] = Excel_Automation.remove_title(y, status = False)
      
      self.removed_title = list_data_nonDuplicate

      list_table = []
      for i in list_data_nonDuplicate:
        table = np.array(i.T.reset_index().values.T.tolist())
        if (Excel_Automation.is_it_table(table,1)):
          list_table.append(i)

      # cek merged cell
      for index,y in enumerate(list_table):
        list_table[index] = Excel_Automation.check_merged_cell(y, status = True)
      
      self.checked_merge_cell = list_table

      list_table_result = []
      for i in list_table:
        table = np.array(y.T.reset_index().values.T.tolist())
        if (Excel_Automation.is_it_table(table,1)):
          list_table_result.append(i)

      return list_table_result

  #====== METHOD FOR PARSING ===========
  def parse_tabel(data):
      data_array = np.array(data.T.reset_index().values.T.tolist())
      data_array_copy = data_array.copy()
      size = data_array.shape

      # convert table into 1 and 0
      for i in range(size[0]):
        for y in range(size[1]):
          if (str(data_array[i,y]) == 'nan' or 'Unnamed' in str(data_array[i,y])):
            data_array[i,y] = 0
          else:
            data_array[i,y] = 1


      list_of_dataframes = []
      l = label(data_array) # labeling tabel 

      region = regionprops(l) # get start and end of row and column

      # parsing besed on region
      for i in range(len(region)):
        firstRow = region[i].bbox[0]
        firstColumn = region[i].bbox[1]  

        if (i < (len(region) - 1)):
          secondRow = region[i+1].bbox[0]
          secondColumn = region[i+1].bbox[1]

          if (firstRow == secondRow):
            if (firstColumn == secondColumn):
              slice_data = data_array_copy[region[i].bbox[0]:region[i+1].bbox[2],region[i].bbox[1]:region[i+1].bbox[3]]
              list_of_dataframes.append(pd.DataFrame(slice_data[1:], columns = slice_data[0]))
            else:
              if (i < (len(region) - 2)):
                thirdRow = region[i+2].bbox[0]
                thirdColumn = region[i+2].bbox[1]
                if (thirdColumn == secondColumn and thirdRow == firstRow):
                  slice_data = data_array_copy[region[i].bbox[0]:region[i+2].bbox[2],region[i].bbox[1]:region[i+2].bbox[3]]
                  list_of_dataframes.append(pd.DataFrame(slice_data[1:], columns = slice_data[0]))
                else:
                  slice_data = data_array_copy[region[i].bbox[0]:region[i+1].bbox[2],region[i].bbox[1]:region[i+1].bbox[3]]
                  list_of_dataframes.append(pd.DataFrame(slice_data[1:], columns = slice_data[0]))
              else:
                slice_data = data_array_copy[region[i].bbox[0]:region[i+1].bbox[2],region[i].bbox[1]:region[i+1].bbox[3]]
                list_of_dataframes.append(pd.DataFrame(slice_data[1:], columns = slice_data[0]))
          else:
            slice_data = data_array_copy[region[i].bbox[0]:region[i].bbox[2],region[i].bbox[1]:region[i].bbox[3]]
            list_of_dataframes.append(pd.DataFrame(slice_data[1:], columns = slice_data[0]))
        else:
          slice_data = data_array_copy[region[i].bbox[0]:region[i].bbox[2],region[i].bbox[1]:region[i].bbox[3]]
          list_of_dataframes.append(pd.DataFrame(slice_data[1:], columns = slice_data[0]))

      list_parseTable = []

      # checking is it table or not
      for i in list_of_dataframes:
        if (Excel_Automation.is_it_table(i,1)):
            list_parseTable.append(i)

      # remove table that contains none value in all cell (error in parsing)
      list_cleanParsing = []
      for i in list_parseTable:
        if (Excel_Automation.remove_none_table(i) == False):
          list_cleanParsing.append(i)
      return list_cleanParsing
  #=============================================================================
  #=============================================================================

  #================== REMOVE UNUSED ROW ========================================
  #=============================================================================

  #======== METHOD FOR CHECKING COLUMN INDEX ==========
  def get_index_column(data):
    list_index_column = []

    # convert value in list data (variable data) to integer
    for i in data:
      try:
        a = int(i)
        list_index_column.append(a)
      except:
        continue
    result = 0
    
    dif_value = []
    for i in range(len(list_index_column)-1):
      dif_value.append(list_index_column[i]-list_index_column[i+1])

    if (len(list_index_column) <= 2): 
      result = False
    elif (Excel_Automation.check_zero_dif(dif_value)):
      result = True
    else:
      different_value = list_index_column[1] - list_index_column[0]
      
      counter = 0
      for i in range(len(list_index_column)-1):
        check = list_index_column[i+1] - list_index_column[i]
        if (check != different_value):
          result = False
          break
        else:
          if ((i+1) == len(list_index_column)-1):
            if (list_index_column[0] >= 1500):
              result = False
            else:
              result = True

    return result

  #============== METHOD FOR CHECKING INDEX COLUMN THAT HAVE DIFFERENT VALUES 0 ===================
  def check_zero_dif(data):
    list_boolean = []
    for i in data:
      if (i == 0):
        list_boolean.append(True)
      else:
        list_boolean.append(False)

    if (False in list_boolean):
      return False
    else:
      return True

  #======== METHOD FOR CHECKING NONE ROW ==========
  def get_none_row(data):
    list_noneData = []
    counter = 0
    
    for i in data:
      if (str(i) == 'nan' or 'Unnamed' in str(i)):
        list_noneData.append(counter)
      counter += 1
    
    if (len(list_noneData) ==  len(data)):
      return True
    else:
      return False

  #========== METHOD FOR DELETE NONE ROW ===========
  def remove_unused_row(data, status = False):
    if (status):
      data_array = np.array(data.T.reset_index().values.T.tolist())
      data_result = []

      for i in range(len(data_array)):
        if (Excel_Automation.get_none_row(data_array[i,:]) or Excel_Automation.get_index_column(data_array[i,:])):
          continue
        else:
          data_result.append(data_array[i,:])

      result = np.array(data_result)

      return pd.DataFrame(result[1:], columns = result[0])
    else:
      return data

  #=========== METHOD FOR CHECKING NONE TABLE AND DELETE IT
  def remove_none_table(table):
    data_array = np.array(table.T.reset_index().values.T.tolist())

    row = data_array.shape[0]
    column = data_array.shape[1]

    counter = 0
    for i in range(row):
      for y in range(column):
        if (data_array[i,y] == None or str(data_array[i,y]) == 'nan' or str(data_array[i,y]) == 'None' or 'Unnamed' in str(data_array[i,y])):
          counter += 1
    
    if (counter == (row*column)):
      return True
    else:
      return False

  #=============================================================================
  #=============================================================================

  #==================== CHECK DUPLICATE TABLE ==================================
  #=============================================================================

  #========== METHOD FOR CHECKING TABLE SIMILARITY BASED ON DATA AT THE TABLE ===============
  def check_similarity_table_2(table1, table2):
    #start = time.time()
    np_array1 = np.array(table1.T.reset_index().values.T.tolist())
    np_array2 = np.array(table2.T.reset_index().values.T.tolist())

    size_1 = np_array1.shape[0]*np_array1.shape[1]
    size_2 = np_array2.shape[0]*np_array2.shape[1]

    #0 brarti tidak ada yang sama
    #1 brarti table_2 in table_1
    #2 brarti table_1 in table_2

    result = 0
    
    if (size_1 > size_2):
      check = Excel_Automation.check_detail_similarity(np_array2,np_array1)
      if (check == 1):
        result = 2
      else:
        result = 0
    elif (size_2 > size_1):
      check = Excel_Automation.check_detail_similarity(np_array1,np_array2)
      if (check == 1):
        result = 1
      else:
        result = 0
    else:
      check = Excel_Automation.check_detail_similarity(np_array1,np_array2)
      if (check == 1):
        result = 1
      else:
        result = 0

    return result

  def check_detail_similarity(table_1, table_2):
    start_value = table_1[0,0]
    size = table_1.shape[1]*table_1.shape[0]
    loc = np.where(table_2 == start_value)
    if (len(loc[0]) == 0 ):
      return 0
    else:
      list_result = []
      for i in range(len(loc[0])):
        selec_data = table_2[loc[0][i]:(loc[0][i]+table_1.shape[0]),loc[1][i]:(loc[1][i]+table_1.shape[1])]
        check_size = selec_data.shape[1]*selec_data.shape[0]
        if (check_size != size):
          list_result.append(False)
        else:
          check_data = np.isin(table_1, selec_data).sum()
          if (check_data == size):
            list_result.append(True)
          else:
            list_result.append(False)
      if (True in list_result):
        return 1
      else:
        return 0

  #========== METHOD FOR REMOVING TITLE ========================================
  #=============================================================================
  def remove_title(data, status=False):
    if (status):
      result = []
      data_array = np.array(data.T.reset_index().values.T.tolist())
      check_title, list_index = 0, 0

      counter = 0
      for i in range(len(data_array)):
        check_title,lis_index = Excel_Automation.check_none_data(data_array[i,:], (len(data_array[i,:])))

        if (check_title):
          if (i < (len(data_array)-1)):
            if (Excel_Automation.matching_row(data_array[i,:], data_array[i+1,:], len(data_array[i,:])-1)):
              if (check_title):
                continue
              else:
                result.append(data_array[i,:])
            else:
              if (i < (len(data_array)-2)):
                if (Excel_Automation.matching_row_3Layer(data_array[i,:], data_array[i+1,:], data_array[i+2,:], len(data_array[i,:])-1)):
                  if (check_title):
                    continue
                  else:
                    result.append(data_array[i,:])
                else:
                  if (i < (len(data_array)-3)):
                    if (Excel_Automation.matching_row_4Layer(data_array[i,:], data_array[i+1,:], data_array[i+2,:], data_array[i+3,:], len(data_array[i,:])-1)):
                      if (check_title):
                        continue
                      else:
                        result.append(data_array[i,:])
                  else:
                    continue
              else:
                continue
          else:
            continue
        else:
          result.append(data_array[i,:])
        counter += 1

      return pd.DataFrame(result[1:], columns = result[0])
    else:
      return data

  #=========== METHOD FOR CHECK TITLE BASED ON NONE VALUE ============
  def check_none_data(list_data, length):
    counter = 0
    list_index = []
    data = 0

    list_index_nonNull = []
    for i in list_data:
      if (str(i) == 'nan' or 'Unnamed' in str(i)):
        list_index.append(counter)
      else:
        list_index_nonNull.append(counter)
      counter += 1

    boolean_result = 0
    
    if ((length-1) in list_index and len(list_data) > 3):
      if ((list_index[-1]-list_index[0]+1) == len(list_index) and (len(list_index_nonNull) > 1)):
        boolean_result = False
      elif ((list_index[-1]-list_index[0]+1) == len(list_index)):
        boolean_result = True
      else:
        boolean_result = False
    elif ((0 in list_index) and len(list_data) > 3):
      if ((list_index[-1]-list_index[0]+1) == len(list_index) and (len(list_index_nonNull) == 1)):
        boolean_result = True
      else:
        boolean_result = False
    else:
      boolean_result = False
    
    return boolean_result, list_index
  #=============================================================================
  #=============================================================================

  #============ METHOD FOR CHECKING MERGE CELL =================================
  #=============================================================================
  def check_merged_cell(data):
    data_array = np.array(data.T.reset_index().values.T.tolist())

    if (len(data_array) > 2):
      if (Excel_Automation.matching_row(data_array[0,:],data_array[1,:], len(data_array[0,:])-1)):

        data_array[0,:] = Excel_Automation.rename_header(data_array[0,:], data_array[1,:])
      
        data_result = np.delete(data_array,1,0)
        return pd.DataFrame(data_result[1:], columns = data_result[0])

      else:
        if (len(data_array) > 3):
          if (Excel_Automation.matching_row_3Layer(data_array[0,:],data_array[1,:], data_array[2,:], len(data_array[0,:])-1)):

            data_array[1,:] = Excel_Automation.rename_header(data_array[1,:], data_array[2,:])
            data_array[0,:] = Excel_Automation.rename_header(data_array[0,:], data_array[1,:])
      
            data_result = np.delete(data_array,[1,2],0)
            return pd.DataFrame(data_result[1:], columns = data_result[0])

          else:
            if (len(data_array) > 4):
              if (Excel_Automation.matching_row_4Layer(data_array[0,:],data_array[1,:], data_array[2,:], data_array[3,:], len(data_array[0,:])-1)):

                data_array[2,:] = Excel_Automation.rename_header(data_array[2,:], data_array[3,:])
                data_array[1,:] = Excel_Automation.rename_header(data_array[1,:], data_array[2,:])
                data_array[0,:] = Excel_Automation.rename_header(data_array[0,:], data_array[1,:])
      
                data_result = np.delete(data_array,[1,2,3],0)
                return pd.DataFrame(data_result[1:], columns = data_result[0])

              else:
                if (Excel_Automation.check_column_merge(data_array[0,:],data_array[1,:])[0]):
                  index = Excel_Automation.check_column_merge(data_array[0,:],data_array[1,:])[1]
                  counter = 1
                  for i in index:
                    data_array[0,i] = str(data_array[0,i-1]) + "_" + str(counter)
                    counter += 1
      
                  data_array[0,:] = Excel_Automation.rename_header(data_array[0,:], data_array[1,:])
      
                  data_result = np.delete(data_array,1,0)
                  return pd.DataFrame(data_result[1:], columns = data_result[0])

                else: 
                  return pd.DataFrame(data_array[1:], columns = data_array[0])
            else:
              return pd.DataFrame(data_array[1:], columns = data_array[0])
        else:
          return pd.DataFrame(data_array[1:], columns = data_array[0])
    else:
      return pd.DataFrame(data_array[1:], columns = data_array[0])

  #========== METHOD FOR CHECKING COLUMN MERGE IN HEADER ==============
  def check_column_merge(data1, data2):
    None1, nonNull1 = Excel_Automation.get_none_index(data1)
    None2, nonNull2 = Excel_Automation.get_none_index(data2)

    list_index_nonCover = []
    for i in None1:
      if (i in None2):
        list_index_nonCover.append(i)
    
    index_merge = []
    for i in None1:
      if (i in nonNull2):
        continue
      else:
        index_merge.append(i)

    list_column_merge = []
    for i in index_merge:
      if (i in list_index_nonCover):
        list_column_merge.append(i)

    if (len(list_column_merge) == 0):
      return False,0
    else:
      return True,list_column_merge

  #=========== METHOD FOR CHECKING TWO ROW ARE MERGE IN HEADER ========
  def matching_row(data1, data2, last_index):
    None1, nonNull1 = Excel_Automation.get_none_index(data1)
    None2, nonNull2 = Excel_Automation.get_none_index(data2)
    list_boolean = []

    list_duplicate_index = []
    list_index_nonCover = []

    for i in nonNull1:
      if (i in nonNull2):
        list_duplicate_index.append(i)
    
    if (0 in list_duplicate_index or last_index in list_duplicate_index):
      list_boolean.append(False)
    else:
      for i in list_duplicate_index:
        if (i == 0):
          if ((i+ 1) in list_duplicate_index):
            list_boolean.append(False)
          else:
            list_boolean.append(True)
        elif (i == last_index):
          if ((i-1) in list_duplicate_index):
            list_boolean.append(False)
          else:
            list_boolean.append(True)
        else:
          if ((i-1) in list_duplicate_index or (i+1) in list_duplicate_index):
            list_boolean.append(False)
          else:
            list_boolean.append(True)
    
    cek1 = 0
    for i in None1:
      if (i in nonNull2):
        cek1 = True
      else:
        cek1 = False
    
    if (cek1):
      list_boolean.append(True)
    else:
      list_boolean.append(False)
    
    for i in None2:
      if (i in nonNull1):
        cek1 = True
      else:
        cek1 = False
    
    if (cek1):
      list_boolean.append(True)
    else:
      list_boolean.append(False)

    if (False in list_boolean):
      return False
    else:
      return True  

  #=========== METHOD FOR CHECKING THREE ROW ARE MERGE IN HEADER ========
  def matching_row_3Layer(data1, data2, data3, last_index):
    None1, nonNull1 = Excel_Automation.get_none_index(data1)
    None2, nonNull2 = Excel_Automation.get_none_index(data2)
    None3, nonNull3 = Excel_Automation.get_none_index(data3)  

    list_index_duplicate = []
    list_index_nonCover = []
    list_boolean = []

    for i in None2:
      if (i in None3):
        list_index_nonCover.append(i)

    for i in nonNull2:
      if (i in nonNull3):
        list_index_duplicate.append(i)

    list_index_inAllThree = []
    for i in list_index_duplicate:
      if (i in nonNull1):
        list_index_inAllThree.append(i)

  # mathcing row 2 layer
    if (0 in list_index_duplicate or last_index in list_index_duplicate):
      list_boolean.append(False)
    else:
      for i in list_index_duplicate:
        if (i == 0):
          if ((i+ 1) in list_index_duplicate):
            list_boolean.append(False)
          else:
            list_boolean.append(True)
        elif (i == last_index):
          if ((i-1) in list_index_duplicate):
            list_boolean.append(False)
          else:
            list_boolean.append(True)
        else:
          if ((i-1) in list_index_duplicate or (i+1) in list_index_duplicate):
            list_boolean.append(False)
          else:
            list_boolean.append(True)

  # mathcing row 3 layer
    if (0 in list_index_inAllThree or last_index in list_index_inAllThree):
      list_boolean.append(False)
    else:  
      for i in list_index_inAllThree:
        if (i == 0):
          if (i+1 in list_index_inAllThree):
            list_boolean.append(False)
          else:
            list_boolean.append(True)
        elif (i == last_index):
          if (i-1 in list_index_inAllThree):
            list_boolean.append(False)
          else:
            list_boolean.append(True)
        else:
          if (i-1 in list_index_inAllThree  or i+1 in list_index_inAllThree):
            list_boolean.append(False)
          else:
            list_boolean.append(True)
    
    for i in list_index_nonCover:
      if (i in None1):
        list_boolean.append(False)
      else:
        list_boolean.append(True)
    
    if (False in list_boolean):
      return False
    else:
      return True

  #=========== METHOD FOR CHECKING FOUR ROW ARE MERGE IN HEADER ========
  def matching_row_4Layer (data1, data2, data3, data4, last_index):
    None1, nonNull1 = Excel_Automation.get_none_index(data1)
    None2, nonNull2 = Excel_Automation.get_none_index(data2)
    None3, nonNull3 = Excel_Automation.get_none_index(data3)
    None4, nonNull4 = Excel_Automation.get_none_index(data4)

    list_index_duplicate = []
    list_index_inAllThree = []
    list_index_inAllFour = []
    list_index_nonCover1 = []
    list_index_nonCover2 = []
    list_boolean = []
    
    for i in None3:
      if (i in None4):
        list_index_nonCover1.append(i)    

    for i in list_index_nonCover1:
      if (i in None2):
        list_index_nonCover2.append(i)
    
    for i in nonNull3:
      if (i in nonNull4):
        list_index_duplicate.append(i)

    for i in list_index_duplicate:
      if (i in nonNull2):
        list_index_inAllThree.append(i)

    for i in list_index_inAllThree:
      if (i in nonNull1):
        list_index_inAllFour.append(i) 

  # mathcing row 2 layer
    if (0 in list_index_duplicate or last_index in list_index_duplicate):
      list_boolean.append(False)
    else:
      for i in list_index_duplicate:
        if (i == 0):
          if ((i+ 1) in list_index_duplicate):
            list_boolean.append(False)
          else:
            list_boolean.append(True)
        elif (i == last_index):
          if ((i-1) in list_index_duplicate):
            list_boolean.append(False)
          else:
            list_boolean.append(True)
        else:
          if ((i-1) in list_index_duplicate or (i+1) in list_index_duplicate):
            list_boolean.append(False)
          else:
            list_boolean.append(True)

  # mathcing row 3 layer
    if (0 in list_index_inAllThree or last_index in list_index_inAllThree):
      list_boolean.append(False)
    else:  
      for i in list_index_inAllThree:
        if (i == 0):
          if (i+1 in list_index_inAllThree):
            list_boolean.append(False)
          else:
            list_boolean.append(True)
        elif (i == last_index):
          if (i-1 in list_index_inAllThree):
            list_boolean.append(False)
          else:
            list_boolean.append(True)
        else:
          if (i-1 in list_index_inAllThree  or i+1 in list_index_inAllThree):
            list_boolean.append(False)
          else:
            list_boolean.append(True)

  # mathcing row 4 layer
    if (0 in list_index_inAllFour or last_index in list_index_inAllFour):
      list_boolean.append(False)
    else:  
      for i in list_index_inAllFour:
        if (i == 0):
          if (i+1 in list_index_inAllFour):
            list_boolean.append(False)
          else:
            list_boolean.append(True)
        elif (i == last_index):
          if (i-1 in list_index_inAllFour):
            list_boolean.append(False)
          else:
            list_boolean.append(True)
        else:
          if (i-1 in list_index_inAllFour or i+1 in list_index_inAllFour):
            list_boolean.append(False)
          else:
            list_boolean.append(True)

    for i in list_index_nonCover2:
      if (i in None1):
        list_boolean.append(False)
      else:
        list_boolean.append(True)

    if (False in list_boolean):
      return False
    else:
      return True  

  #========= METHOD FOR GET INDEX FOR NONE VALUE ==========
  def get_none_index(list_data):
    counter = 0 
    list_index_none = []
    list_index_nonNull = []
    for i in list_data:
      if (str(i) == 'nan' or 'Unnamed' in str(i)):
        list_index_none.append(counter)
      else:
        list_index_nonNull.append(counter)
      counter += 1
    return list_index_none, list_index_nonNull

  #=========== METHOD FOR RENAME TABLE HEADER AFTER CHECKING MERGE CELL ==============
  def rename_header(data1, data2):
    index_null1, index_nonNull1 = Excel_Automation.get_none_index(data1)
    index_null2, index_nonNull2 = Excel_Automation.get_none_index(data2)

    duplicate_index = []
    data = data1.copy()

    for i in index_nonNull1:
      if (i in index_nonNull2):
        duplicate_index.append(i)

    for i in range(len(data1)):
      if (i in duplicate_index):
        data[i] = str(data2[i]) + "_" + str(data1[i]) 
      elif ((i in index_null1) and (i in index_null2)):
        continue
      elif (i in index_null1):
        index = Excel_Automation.find_index_merged_cell(duplicate_index, i)
        data[i] = str(data2[i]) + "_" + str(data1[index])
      else:
        continue
    
    return data

  #========== METHOD FOR HELPING IN RENAME TABLE HEADER =========
  def find_index_merged_cell(duplicate_index, index):
    for i in range(len(duplicate_index)):
      if (index == duplicate_index[i]):
        return index
        break
      else:
        if (i != (len(duplicate_index)-1)):
          if (index > duplicate_index[i] and index < duplicate_index[i+1]):
            return duplicate_index[i]
          else:
            continue
        else:
          if (index > duplicate_index[i]):
            return duplicate_index[i]

  #=============================================================================
  #=============================================================================

  #====== METHOD FOR CHECK IT TABLE OR NOT =====================================
  #=============================================================================
  def is_it_table(data, max_row):
      row,column = data.shape

      if (row < max_row):
        return False
      elif (row <= 2 and column <= 2):
        return False
      else:
        return True

time: 8.67 s (started: 2021-03-10 08:41:00 +00:00)


In [51]:
excel = 0
#excel = Excel_Automation('/content/dataTPA.xlsx', list_function = '[parse_table,check_duplicate_table,remove_title,check_merge_cell]')
excel = Excel_Automation('/content/dataTPA.xlsx', list_function = '[remove_unused_row]')

time: 92.6 ms (started: 2021-03-10 08:41:31 +00:00)


In [42]:
excel.result_parsing

AttributeError: ignored

time: 15.4 ms (started: 2021-03-10 07:51:13 +00:00)


In [19]:
df = pd.read_excel('/content/dataTPA.xlsx', sheet_name='Lembar1', engine='openpyxl')
a = np.array(df.T.reset_index().values.T.tolist())

In [36]:
c = 0
for i in df:
  c += 1
  if c == 3:
    break
  else:
    print(i)

DATA PROFIL SANTRI
Unnamed: 1


In [34]:
a = 1
print(type(a) is list)

False
